여기서 보자!! 음 eos 토큰은 안쓰이는 숫자 여야 한다.


concepts에 3~126이라 할때 
response가 0,1이고 

concepts도 eos가, response에도 eos가 있어야 하겠지?
concepts+123*response = 이렇게 되면 기존에는 0~122 까지의 값에 0,123 이 더해지는 것이었잖아

오답일 때 가질 수 있는 값은 0~122 
정답일 떄 가질 수 있는 값은 123 ~225


-----------------------------------------------------
concepts을 3~125까지
response를 0,1 이라 할때
concepts+123*response =

Interaction:
오답일 때 가질 수 있는 값은 3~125 
정답일 떄 가질 수 있는 값은 126 ~228

Question:
3~125

;;;;;;;;;;;;
eos =2 일때

Interaction:
오답일 때 가질 수 있는 값은 3~125 + eos 토큰끼리 더해졌을 때 4 추가 --> 곂침
정답일 떄 가질 수 있는 값은 126 ~228 +eos 토큰끼리 더해졌을 때 4

--> 이 경우 r에 더해진 eos값이 123이 곱해지며 또 달리 질 수 잇기에 r값에 eos를 더해주기 전에 미리 123(concept의)숫 자를 구한것을 따로 곱해두고 거기에 eos를 추가하자 
r_shift경우 r을 두개로 만들어서 하나는 interaction용으로 빼두고 하나는 y값으로 쓰이게 eos만 추가해서 따로 두자!

->거기다가 c,r에는 eos/2 를 더해 두개가 더해졌을 때 eos가 되도록 하고 
c_shift, r_shift에는 온전한 eos를 더해두는 거지 그러면 문제 해결?

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import json
import copy
import itertools
import tensorflow as tf
from preprocess.data_utils import read_data,get_max_concepts,calStatistics,extend_multi_concepts,id_mapping,save_id2idx,train_test_split,save_dcur,get_evalmask_token, get_mask_tokens
import pickle
import glob
import matplotlib.pyplot as plt
import time
from models.model_for_kt import TFTransfoXLModel,TFTransfoXLLMHeadModel,TFTransfoXLMLMHeadModel
from tqdm import tqdm
from transformers import TransfoXLConfig
from tensorflow.keras.utils import register_keras_serializable
import datetime
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"



2024-01-18 16:27:05.676426: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-18 16:27:06.206847: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:
2024-01-18 16:27:06.206906: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:
2024-01-18 16:27:06.206924: W tensorflow/compiler/tf2tensorrt/utils/py_uti

In [2]:

config_xl = TransfoXLConfig(
    data = '/home/jun/workspace/KT/data/ednet/data.txt',
    dataset = 'wt103',
    d_embed=128,
    d_head = 32,
    d_model=128,
    mem_len=200,
    n_head=8,
    n_layer=6,
    batch_size = 65,
    tgt_len = 140,
    ext_len = 0,
    eval_tgt_len = 36,
    eos_token=2,
    num_c=123,
    mask_token=3,
    C_vocab_size=188,
    Q_vocab_size = 12277,
    R_vocab_size = 2,
    epoch = 1,
    tf_dir ='/home/jun/workspace/KT/data/ednet/TF_DATA',
    mode = 'concepts' # questions
)

In [3]:





# ALL_KEYS = ["fold", "uid", "questions", "concepts", "responses", "timestamps",
#             "usetimes", "selectmasks", "is_repeat", "qidxs", "rest", "orirow", "cidxs"]
# ONE_KEYS = ["fold", "uid"]

total_df, effective_keys = read_data('/home/jun/workspace/KT/data/ednet/data.txt')

stares = []

if 'concepts' in effective_keys:
    max_concepts = get_max_concepts(total_df)
else:
    max_concepts = -1

oris, _, qs, cs, seqnum = calStatistics(total_df, stares, "original") # Check data status

print("="*20)
print(
    f"original total interactions: {oris}, qs: {qs}, cs: {cs}, seqnum: {seqnum}")


# questions ,concepts 값들의 숫자를 재정의 하여 0~ 나오도록 만들 면서 is_repeat값 처리
# 14_54_14 으로 된 concepts를 14,54,14 로 분리하고 기존의 response도 똑같이 확장처리
total_df_ex, effective_keys = extend_multi_concepts(total_df, effective_keys)
total_df, dkeyid2idx = id_mapping(total_df_ex)
dkeyid2idx["max_concepts"] = max_concepts

extends, _, qs, cs, seqnum = calStatistics(
    total_df, stares, "extend multi")
print("="*20)
print(
    f"after extend multi, total interactions: {extends}, qs: {qs}, cs: {cs}, seqnum: {seqnum}")


#train test 분리
train_df, test_df = train_test_split(total_df, 0.2)

#transformer_xl dataset 만들기
train = {"qseqs": [], "cseqs": [], "masked_R": [], "labels": []} #"q_shift": [], "c_shift": [], "r_shift": []}
test = {"qseqs": [], "cseqs": [], "masked_R": [], "labels": []} #"q_shift": [], "c_shift": [], "r_shift": []}


delete bad stu num of len: 7, delete interactions: 11, of r: 0, good num: 6162174
original total interactions: 6162174, qs: 12102, cs: 188, seqnum: 4990
df.columns: Index(['uid', 'is_repeat', 'concepts', 'timestamps', 'responses', 'usetimes',
       'questions'],
      dtype='object')
after extend multi, total interactions: 15220330, qs: 12102, cs: 188, seqnum: 4990
total num: 4990, train+valid num: 3992, test num: 998


In [4]:

# eos_token= 2 # config 파일로 조정할 수 있도록 수정 할 것!
# num_c =123 # config 파일로 조정할 수 있도록 수정 할 것!
for i, row in train_df.iterrows():
#use kc_id or question_id as input

    cseq_list=[int(_) for _ in row["concepts"].split(",")]
    cseq_list.append(config_xl.eos_token)
    train["cseqs"].append(cseq_list)

    qes_list=[int(_) for _ in row["questions"].split(",")]
    qes_list.append(config_xl.eos_token)
    train["qseqs"].append(qes_list)


    rseq_list=[(int(_)) for _ in row["responses"].split(",")]
    rseq_list.append(config_xl.eos_token)
    masked_R, labels = get_mask_tokens(rseq_list,config_xl.mask_token, config_xl.eos_token)

    train["masked_R"].append(masked_R)
    train["labels"].append(labels)
for i, row in test_df.iterrows():
    #use kc_id or question_id as input
    
    cseq_list=[int(_) for _ in row["concepts"].split(",")]
    cseq_list.append(config_xl.eos_token)   
    test["cseqs"].append(cseq_list)

    qes_list=[int(_) for _ in row["questions"].split(",")]
    qes_list.append(config_xl.eos_token)
    test["qseqs"].append(qes_list)


    rseq_list=[(int(_)) for _ in row["responses"].split(",")]
    rseq_list.append(config_xl.eos_token)
    masked_R, labels = get_evalmask_token(rseq_list, config_xl.mask_token, config_xl.eos_token)
    

    test["masked_R"].append(masked_R)
    test["labels"].append(labels)

2024-01-18 16:27:39.454610: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-18 16:27:39.454944: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-18 16:27:39.460372: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-18 16:27:39.460717: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-18 16:27:39.460898: I tensorflow/compiler/xla/stream_executo

In [5]:
#with tf.device("/gpu:0"):
 #       with open('/home/jun/workspace/KT/data/ednet/train.pkl', "rb") as file:
  #             train = pickle.load(file) 
               
# with open('/home/jun/workspace/KT/data/ednet/test.pkl', "rb") as file:
#         test = pickle.load(file) 

In [6]:
#train
cseqs_list = list(itertools.chain(*train['cseqs']))
qseqs_list = list(itertools.chain(*train['qseqs']))
r_masked_list = tf.concat(train['masked_R'], axis=0)
labels = tf.concat(train['labels'], axis=0)

n_step = len(cseqs_list) // (config_xl.batch_size*config_xl.tgt_len)





In [7]:

sliced_cseqs = tf.slice(cseqs_list,[0],[n_step * config_xl.batch_size*config_xl.tgt_len])  
sliced_qseqs = tf.slice(qseqs_list,[0],[n_step * config_xl.batch_size*config_xl.tgt_len])  
sliced_r_mask = tf.slice(r_masked_list,[0],[n_step * config_xl.batch_size*config_xl.tgt_len]) 
sliced_labels = tf.slice(labels,[0],[n_step * config_xl.batch_size*config_xl.tgt_len]) 

count =len(sliced_cseqs)// (config_xl.batch_size*config_xl.tgt_len)

new_shape = (config_xl.batch_size, -1)  # 나머지 차원은 자동으로 계산됨

cseq_reshaped = tf.reshape(sliced_cseqs, new_shape)
qseq_reshaped = tf.reshape(sliced_qseqs, new_shape)
r_mask_reshaped = tf.reshape(sliced_r_mask, new_shape)
labels_reshaped = tf.reshape(sliced_labels, new_shape)

cseq_transposed = tf.transpose(cseq_reshaped)
qseq_transposed = tf.transpose(qseq_reshaped)
r_mask_transposed = tf.transpose(r_mask_reshaped)
labels_transposed = tf.transpose(labels_reshaped)


In [8]:

#test
test_cseqs_list = list(itertools.chain(*test['cseqs']))
test_qseqs_list = list(itertools.chain(*test['qseqs']))
test_r_masked_list = tf.concat(test['masked_R'], axis=0)
test_labels = tf.concat(test['labels'], axis=0)



# Work out how cleanly we can divide the dataset into bsz parts.
# 아래의 두 코드는   data 텐서에서 배치 크기 bsz로 깔끔하게 맞지 않는 추가 요소를 제거하는 것 배치에 띡 떨어지게
    
test_n_step = len(test_cseqs_list) // (config_xl.batch_size*config_xl.tgt_len)
print('n_step',test_n_step) # 

test_sliced_cseqs = tf.slice(test_cseqs_list,[0],[test_n_step * config_xl.batch_size*config_xl.tgt_len])  
test_sliced_qseqs = tf.slice(test_qseqs_list,[0],[test_n_step * config_xl.batch_size*config_xl.tgt_len])  
test_sliced_r_masked_seq = tf.slice(test_r_masked_list,[0],[test_n_step * config_xl.batch_size*config_xl.tgt_len]) 

test_sliced_labels = tf.slice(test_labels,[0],[test_n_step * config_xl.batch_size*config_xl.tgt_len])  

count =len(test_sliced_cseqs)// (config_xl.batch_size*config_xl.tgt_len)
print(count)

'''# 시작 위치와 슬라이싱할 크기 설정
begin = [0]  # 첫 번째 차원의 시작 위치는 0
size = [6]   # 첫 번째 차원에서 6개의 원소를 슬라이싱

# 데이터를 잘라내기 (tf.slice 사용)
sliced_data = tf.slice(data, begin, size)  '''

# Evenly divide the da
# ta across the bsz batches.

new_shape = (config_xl.batch_size, -1)  # 나머지 차원은 자동으로 계산됨

test_qseq_reshaped = tf.reshape(test_sliced_qseqs, new_shape)
test_cseq_reshaped = tf.reshape(test_sliced_cseqs, new_shape)
test_r_masked_seq_reshaped = tf.reshape(test_sliced_r_masked_seq, new_shape)
test_labels_reshaped = tf.reshape(test_sliced_labels, new_shape)


test_qseq_reshaped = tf.transpose(test_qseq_reshaped)
test_cseq_reshaped = tf.transpose(test_cseq_reshaped)
test_r_masked_seq_reshaped = tf.transpose(test_r_masked_seq_reshaped)
test_labels_reshaped = tf.cast(tf.transpose(test_labels_reshaped),tf.int64)

n_step 342
342


In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices(
    (qseq_transposed,cseq_transposed, r_mask_transposed, labels_transposed))
train_dataset =train_dataset.batch(config_xl.tgt_len)
# tf.data.experimental.save(train_dataset, config_xl.tf_dir+'/train')


In [10]:

test_dataset = tf.data.Dataset.from_tensor_slices(
    (test_qseq_reshaped,test_cseq_reshaped, test_r_masked_seq_reshaped, test_labels_reshaped))
test_dataset =test_dataset.batch(config_xl.tgt_len)
# tf.data.experimental.save(test_dataset, config_xl.tf_dir+'/test')


In [11]:
# train_dataset = tf.data.experimental.load(config_xl.tf_dir+'/train')


In [12]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = '/home/jun/workspace/KT/logs/gradient_tape/' + current_time +'{}ep_{}mem_Question/train'.format(config_xl.epoch, config_xl.mem_len)
test_log_dir = '/home/jun/workspace/KT/logs/gradient_tape/' + current_time +'{}ep_{}mem_Question/test'.format(config_xl.epoch, config_xl.mem_len)
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [13]:
@register_keras_serializable()
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = tf.cast(d_model, tf.float32)

        self.warmup_steps = tf.cast(warmup_steps,tf.float32)

    def __call__(self, step):
        step =tf.cast(step, tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
    
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)
    
    def get_config(self):
        return {
            'd_model': self.d_model,
            'warmup_steps': self.warmup_steps
            }
learning_rate = CustomSchedule(config_xl.d_model)

In [14]:
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)


In [15]:
model = TFTransfoXLMLMHeadModel(config=config_xl)


In [16]:
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
      name='train_accuracy')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
  name='test_accuracy')
test_precision = tf.metrics.Precision()
test_recall = tf.metrics.Recall()
train_auc = tf.keras.metrics.AUC()
test_auc = tf.keras.metrics.AUC()


In [17]:


@tf.function
def train_step(model, data1,data2, target, mems, optimizer):
    with tf.GradientTape() as tape:
        outputs = model(concepts=data1,responses=data2, labels=target, mems=mems)
        loss = outputs.loss
        mems = outputs.mems
        loss_mx = target != -100
        loss_value = loss[loss_mx]
        loss_value = tf.reshape(loss_value, [-1, config_xl.R_vocab_size])
        labels = target[loss_mx]

        
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=loss_value)
        # batch_loss = tf.reduce_sum(loss) / valid_samples
        mean_loss = tf.reduce_mean(loss)
        train_loss(loss)
        train_accuracy(labels,loss_value)
        predictions =tf.nn.softmax(loss_value)
        train_auc(tf.one_hot(labels, depth=predictions.shape[1]), predictions)

    gradients = tape.gradient(mean_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    

    return mems,mean_loss

# def grad(model, inputs, targets):
#   with tf.GradientTape() as tape:
#     loss_value = loss(model, inputs, targets, training=True)
#   return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [18]:
def evaluate(model, mems, test_dataset):
    total_loss = 0.0
    num_batches = 0
    evaluation_metrics = []
    

    for test_data1, test_data2, test_target in test_dataset:
        outputs = model(concepts=test_data1, responses=test_data2, labels=test_target, mems=mems, training=False)
        loss = outputs.loss
        mems = outputs.mems

        loss_mx = test_target != -100
        loss_value = loss[loss_mx]
        loss_value = tf.reshape(loss_value, [-1, config_xl.R_vocab_size])
        labels = test_target[loss_mx]

        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=loss_value)
        mean_loss = tf.reduce_mean(loss)

        # Update precision and recall metrics
        predicted_labels = tf.argmax(loss_value, axis=1)
        predictions =tf.nn.softmax(loss_value)

        
        test_auc(tf.one_hot(labels, depth=predictions.shape[1]), predictions)
        test_precision(labels, predicted_labels)
        test_recall(labels, predicted_labels)

        test_accuracy(labels, loss_value)
        test_loss(loss)
        
        
        precision = test_precision.result().numpy()
        recall = test_recall.result().numpy()
        f1_score = 2 * (precision * recall) / (precision + recall + 1e-7)

        evaluation_metrics.append(test_accuracy.result().numpy())

        total_loss += mean_loss.numpy()
        num_batches += 1

        
        with test_summary_writer.as_default():
            tf.summary.scalar('loss', test_loss.result(), step=num_batches)
            tf.summary.scalar('accuracy', test_accuracy.result(), step=num_batches)
            tf.summary.scalar('precision', test_precision.result(), step=num_batches)
            tf.summary.scalar('recall', test_recall.result(), step=num_batches)
            tf.summary.scalar('f1_score', f1_score, step=num_batches)
            tf.summary.scalar('auc', test_auc.result(), step=num_batches)

    # 평균 정밀도, 재현율, F1 점수를 계산합니다.
    average_precision = test_precision.result().numpy()
    average_recall = test_recall.result().numpy()
    average_f1_score = 2 * (average_precision * average_recall) / (average_precision + average_recall + 1e-7)

    average_metric = sum(evaluation_metrics) / len(evaluation_metrics)
    average_loss = total_loss / num_batches

    return average_loss, average_metric, average_precision, average_recall, average_f1_score


In [19]:
loss_values = []
num_batches = 0
# tf.summary.trace_on(graph=True, profiler=True)

for epoch in range(config_xl.epoch):
    start = time.time()
    total_loss = 0.0
    mems = None              # 첫 번째 모델의 메모리 상태
           
    for question,ceq, mask, labels in tqdm(train_dataset):
        input_data = ceq if config_xl.mode == 'concepts' else question
        mems, loss_value = train_step(model, input_data,mask, labels, mems, optimizer)
        num_batches += 1
        total_loss += loss_value.numpy()
        if num_batches % 100 == 0:
            loss_values.append(loss_value.numpy())
            print(f'Epoch {epoch + 1} Batch {num_batches} Loss {loss_value.numpy()}')
        with train_summary_writer.as_default():
            tf.summary.scalar('loss', train_loss.result(), step=num_batches)
            tf.summary.scalar('accuracy', train_accuracy.result(), step=num_batches)
            tf.summary.scalar('auc', train_auc.result(), step=num_batches)

    # with writer.as_default():# 텐서보드에 모델 그래프 그리기
    #     tf.summary.trace_export(
    #         name="my_func_trace",
    #         step=0,
    #         profiler_outdir=logdir)             

    # Reset metrics every epoch
    # train_loss.reset_states()
    # test_loss.reset_states()
    # train_accuracy.reset_states()
    # test_accuracy.reset_states()

  0%|          | 0/1331 [00:00<?, ?it/s]

func <class 'method'>
config <class 'transformers.models.transfo_xl.configuration_transfo_xl.TransfoXLConfig'>
kwargs <class 'dict'>
func <class 'method'>
config <class 'transformers.models.transfo_xl.configuration_transfo_xl.TransfoXLConfig'>
kwargs <class 'dict'>
func <class 'method'>
config <class 'transformers.models.transfo_xl.configuration_transfo_xl.TransfoXLConfig'>
kwargs <class 'dict'>
func <class 'method'>
config <class 'transformers.models.transfo_xl.configuration_transfo_xl.TransfoXLConfig'>
kwargs <class 'dict'>


2024-01-18 16:28:15.533935: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x5572443c0a80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-18 16:28:15.533953: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 SUPER, Compute Capability 7.5
2024-01-18 16:28:15.533972: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): NVIDIA GeForce RTX 2080 SUPER, Compute Capability 7.5
2024-01-18 16:28:15.537433: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-18 16:28:15.615873: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
  0%|          | 1/1331 [00:12<4:36:19, 12.47s/it]

func <class 'method'>
config <class 'transformers.models.transfo_xl.configuration_transfo_xl.TransfoXLConfig'>
kwargs <class 'dict'>
func <class 'method'>
config <class 'transformers.models.transfo_xl.configuration_transfo_xl.TransfoXLConfig'>
kwargs <class 'dict'>


  1%|          | 10/1331 [00:19<41:59,  1.91s/it] 


KeyboardInterrupt: 

In [16]:
# test_mems = None
# test_loss0,test_acc0,average_precision, average_recall, average_f1_score = evaluate(model,test_mems, test_dataset)
# print(f'Test Loss on First Half Dataset after Epoch {epoch + 1}: {test_loss0}')

In [17]:
model.save_weights('/home/jun/workspace/KT/save_model/5ep_600mem_questiion.ckpt/my_checkpoint') 
# model.save('/home/jun/workspace/KT/save_model/test')

# model.load_weights('/home/jun/workspace/KT/save_model/5ep_600mem.ckpt/my_checkpoint')



inputs["concepts"] {'mems': None, 'head_mask': None, 'inputs_embeds': None, 'output_attentions': False, 'output_hidden_states': False, 'return_dict': True, 'training': False, 'concepts': <tf.Tensor 'args_0:0' shape=(None, 65) dtype=int32>, 'responses': <tf.Tensor 'responses:0' shape=(None, 65) dtype=int32>, 'labels': None}
inputs Tensor("args_0:0", shape=(None, 65), dtype=int32)
inputs["concepts"] {'mems': None, 'head_mask': None, 'inputs_embeds': None, 'output_attentions': False, 'output_hidden_states': False, 'return_dict': True, 'training': False, 'concepts': <tf.Tensor 'concepts:0' shape=(None, 65) dtype=int32>, 'responses': <tf.Tensor 'responses:0' shape=(None, 65) dtype=int32>, 'labels': None}
input_concepts Tensor("strided_slice:0", shape=(65,), dtype=int32)
inputs Tensor("Reshape:0", shape=(1, 65), dtype=int32)
input_concepts Tensor("strided_slice:0", shape=(65,), dtype=int32)
inputs Tensor("Reshape:0", shape=(1, 65), dtype=int32)
inputs["concepts"] {'mems': None, 'head_mask': 

ValueError: in user code:

    File "/home/jun/workspace/KT/models/model_for_kt.py", line 1335, in serving  *
        output = self.call(inputs)
    File "/home/jun/workspace/KT/models/model_for_kt.py", line 1802, in call  *
        bsz, tgt_len = shape_list(inputs["inputs_embeds"])[:2]
    File "/home/jun/miniconda3/envs/new/lib/python3.8/site-packages/transformers/tf_utils.py", line 39, in shape_list  *
        dynamic = tf.shape(tensor)

    ValueError: None values not supported.


In [ ]:
import tensorflow as tf

# 배치 크기를 가변적으로 처리하려면 첫 번째 차원을 None으로 두고,
tensor = tf.keras.Input(shape=(2,))


In [ ]:
print(tensor.shape[1])


2


In [ ]:
print(test_loss0)
print(test_acc0)
print(average_f1_score)


0.6319265374314715
0.6484004206824721
0.7520016045546908


###임베딩 데이터 시각화

In [ ]:
from tensorboard.plugins import projector

log_dir='/home/jun/workspace/KT/logs/question/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [ ]:
# Save Labels separately on a line-by-line manner.
with open(os.path.join(log_dir, 'metadata.tsv'), "w") as f:
  for concepts in dkeyid2idx['questions']:
    f.write("{}\n".format(concepts))
 

In [ ]:
weights = tf.Variable(model.transformer.word_emb_C.get_weights()[0])


In [ ]:
checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

'/home/jun/workspace/KT/logs/question/embedding.ckpt-1'

In [ ]:
# Set up config.
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)

In [ ]:
#tensorboard --logdir /logs/imdb-example/  실행코드
#체크포인트 저장하는 방법 공부해보자!

In [ ]:
import inspect

# Sample function to demonstrate the use of inspect.signature
def sample_function(arg1, arg2="default", *args, **kwargs):
    print("Sample Function Executed")



In [ ]:
A=dict(inspect.signature(sample_function).parameters)

In [ ]:
B=A.pop('arg1',None)

In [ ]:
B

<Parameter "arg1">

In [3]:
for i in range(20):
    if i % 2 ==0:
        i+100
    else:
        i+10
    
    A=i+1

    print(A)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
